# Cross-Country Validation of Iusmorfos Framework
## External Validation Across Legal Traditions and Development Levels

**Author**: Iusmorfos Research Team  
**Date**: 2024  
**Version**: 1.0

This notebook provides comprehensive cross-country validation of the Iusmorfos framework using datasets from Chile, South Africa, Sweden, and India, comparing against the baseline Argentina dataset.

## Research Questions
1. Does the Iusmorfos framework generalize across different legal traditions (Common Law, Civil Law, Mixed)?
2. How does institutional development level affect framework applicability?
3. Are power-law patterns (γ≈2.3) consistent across countries?
4. What factors drive cross-country variation in legal innovation patterns?

## Table of Contents
1. [Setup and Data Loading](#setup)
2. [Country Dataset Generation](#generation)
3. [Individual Country Validation](#validation)
4. [Cross-Country Comparative Analysis](#comparison)
5. [Legal Tradition Analysis](#legal-tradition)
6. [Development Level Analysis](#development)
7. [Power-Law Universality Testing](#power-law)
8. [Institutional Factors Analysis](#institutional)
9. [Framework Generalizability Assessment](#generalizability)
10. [Conclusions and Policy Implications](#conclusions)

In [ ]:
# Environment setup
import sys
import os
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path('../src').resolve()))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy import stats
from scipy.stats import mannwhitneyu, kruskal, chi2_contingency
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

# Import our modules
from config import get_config
from external_validation import ExternalValidationFramework

# Plotting configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11

print("🌍 Cross-Country Validation Notebook Initialized")
print(f"📊 Timestamp: {pd.Timestamp.now()}")

## 1. Setup and Data Loading {#setup}

In [ ]:
# Initialize validation framework
validator = ExternalValidationFramework()
config = get_config()

print("🔧 Validation framework initialized")
print(f"\n🌍 Countries in validation framework:")

for code, metadata in validator.country_metadata.items():
    print(f"  {code}: {metadata['name']}")
    print(f"      Legal tradition: {metadata['legal_tradition']}")
    print(f"      Development: {metadata['development_level']}")
    print(f"      Region: {metadata['region']}")
    print(f"      Rule of Law Index: {metadata['rule_of_law_index']}")
    print(f"      GDP per capita: ${metadata['gdp_per_capita_2020']:,}")
    print()

## 2. Country Dataset Generation {#generation}

Generate realistic legal innovation datasets for each country based on their institutional characteristics.

In [ ]:
print("📊 GENERATING COUNTRY DATASETS")
print("=" * 40)

# Generate datasets for all countries
countries_to_test = ['AR', 'CL', 'ZA', 'SE', 'IN']
country_datasets = {}

for country_code in countries_to_test:
    print(f"\nGenerating dataset for {validator.country_metadata[country_code]['name']}...")
    dataset = validator.generate_country_dataset(country_code)
    country_datasets[country_code] = dataset
    
    print(f"  Dataset size: {len(dataset):,} innovations")
    print(f"  Time period: {dataset['year'].min()}-{dataset['year'].max()}")
    print(f"  Reform types: {dataset['reform_type'].nunique()}")
    print(f"  Crisis periods: {dataset['during_crisis'].sum()} innovations during crises")

# Create combined dataset for analysis
combined_dataset = pd.concat(country_datasets.values(), ignore_index=True)

print(f"\n✅ Generated datasets for {len(countries_to_test)} countries")
print(f"Combined dataset: {len(combined_dataset):,} total innovations")

# Display dataset overview
print(f"\n📋 Dataset Overview by Country:")
country_summary = combined_dataset.groupby(['country']).agg({
    'complexity_score': ['count', 'mean', 'std'],
    'adoption_success': ['mean', 'std'],
    'citation_count': ['sum', 'mean', 'max'],
    'during_crisis': 'sum'
}).round(3)

print(country_summary)

## 3. Individual Country Validation {#validation}

Validate each country dataset against Iusmorfos framework predictions.

In [ ]:
print("🧪 INDIVIDUAL COUNTRY VALIDATION")
print("=" * 40)

# Validate each country
validation_results = {}

for country_code, dataset in country_datasets.items():
    print(f"\nValidating {validator.country_metadata[country_code]['name']}...")
    result = validator.validate_country_dataset(country_code, dataset)
    validation_results[country_code] = result
    
    # Display key results
    status = "✅ PASSED" if result.validation_passed else "❌ FAILED"
    print(f"  Validation: {status}")
    print(f"  Compatibility score: {result.iusmorfos_compatibility:.3f}")
    print(f"  Power-law γ: {result.power_law_gamma:.3f} (R² = {result.power_law_r2:.3f})")
    print(f"  Complexity mean: {result.complexity_stats['mean']:.2f}")
    print(f"  Adoption success: {result.adoption_stats['mean']:.3f}")
    
    # Display key notes
    if result.notes:
        print(f"  Key findings:")
        for note in result.notes[-3:]:  # Show last 3 notes
            print(f"    {note}")

# Create validation summary
validation_summary = pd.DataFrame([
    {
        'country': result.country_name,
        'code': result.country_code,
        'dataset_size': result.dataset_size,
        'compatibility': result.iusmorfos_compatibility,
        'validation_passed': result.validation_passed,
        'power_law_gamma': result.power_law_gamma,
        'power_law_r2': result.power_law_r2,
        'complexity_mean': result.complexity_stats['mean'],
        'adoption_mean': result.adoption_stats['mean']
    }
    for result in validation_results.values()
])

print(f"\n📊 Validation Summary:")
print(validation_summary.round(3))

# Overall validation rate
overall_rate = validation_summary['validation_passed'].mean()
print(f"\n🎯 Overall validation rate: {overall_rate:.1%}")
if overall_rate >= 0.8:
    print("✅ Strong cross-country validation - Framework generalizes well")
elif overall_rate >= 0.6:
    print("📊 Moderate cross-country validation - Framework shows promise") 
else:
    print("⚠️ Weak cross-country validation - Framework needs refinement")

## 4. Cross-Country Comparative Analysis {#comparison}

Compare patterns across countries and analyze systematic differences.

In [ ]:
# Perform cross-country analysis
cross_analysis = validator.cross_country_comparative_analysis()

# Create comprehensive visualization
fig, axes = plt.subplots(3, 3, figsize=(20, 18))

# Add country metadata to combined dataset
metadata_df = pd.DataFrame.from_dict(validator.country_metadata, orient='index')
metadata_df['country_code'] = metadata_df.index

analysis_df = combined_dataset.merge(
    metadata_df[['country_code', 'name', 'legal_tradition', 'development_level', 'rule_of_law_index', 'gdp_per_capita_2020']], 
    left_on='country', right_on='country_code', how='left'
)

# 1. Complexity distribution by country
sns.boxplot(data=analysis_df, x='name', y='complexity_score', ax=axes[0, 0])
axes[0, 0].set_title('Legal Innovation Complexity by Country')
axes[0, 0].set_xlabel('Country')
axes[0, 0].set_ylabel('Complexity Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Adoption success by country
sns.boxplot(data=analysis_df, x='name', y='adoption_success', ax=axes[0, 1])
axes[0, 1].set_title('Adoption Success by Country')
axes[0, 1].set_xlabel('Country')
axes[0, 1].set_ylabel('Adoption Success Rate')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Citation distribution by country (log scale)
analysis_df['log_citations'] = np.log1p(analysis_df['citation_count'])
sns.boxplot(data=analysis_df, x='name', y='log_citations', ax=axes[0, 2])
axes[0, 2].set_title('Citation Distribution by Country (log scale)')
axes[0, 2].set_xlabel('Country')
axes[0, 2].set_ylabel('Log(Citations + 1)')
axes[0, 2].tick_params(axis='x', rotation=45)

# 4. Complexity by legal tradition
sns.boxplot(data=analysis_df, x='legal_tradition', y='complexity_score', ax=axes[1, 0])
axes[1, 0].set_title('Complexity by Legal Tradition')
axes[1, 0].set_xlabel('Legal Tradition')
axes[1, 0].set_ylabel('Complexity Score')
axes[1, 0].tick_params(axis='x', rotation=45)

# 5. Adoption success by development level
sns.boxplot(data=analysis_df, x='development_level', y='adoption_success', ax=axes[1, 1])
axes[1, 1].set_title('Adoption Success by Development Level')
axes[1, 1].set_xlabel('Development Level')
axes[1, 1].set_ylabel('Adoption Success Rate')

# 6. Rule of law vs adoption success (country level)
country_means = analysis_df.groupby(['name', 'rule_of_law_index']).agg({
    'adoption_success': 'mean',
    'complexity_score': 'mean'
}).reset_index()

axes[1, 2].scatter(country_means['rule_of_law_index'], country_means['adoption_success'], 
                  s=100, alpha=0.7)
for i, row in country_means.iterrows():
    axes[1, 2].annotate(row['name'], (row['rule_of_law_index'], row['adoption_success']),
                       xytext=(5, 5), textcoords='offset points', fontsize=9)

# Add trend line
z = np.polyfit(country_means['rule_of_law_index'], country_means['adoption_success'], 1)
p = np.poly1d(z)
axes[1, 2].plot(country_means['rule_of_law_index'], p(country_means['rule_of_law_index']), 
               "r--", alpha=0.8)

axes[1, 2].set_title('Rule of Law vs Adoption Success')
axes[1, 2].set_xlabel('Rule of Law Index')
axes[1, 2].set_ylabel('Mean Adoption Success')
axes[1, 2].grid(True, alpha=0.3)

# 7. Power-law gamma by country
gamma_data = [(result.country_name, result.power_law_gamma) 
              for result in validation_results.values() if result.power_law_gamma > 0]

if gamma_data:
    gamma_df = pd.DataFrame(gamma_data, columns=['country', 'gamma'])
    axes[2, 0].bar(gamma_df['country'], gamma_df['gamma'], alpha=0.7)
    axes[2, 0].axhline(y=2.3, color='red', linestyle='--', label='Expected γ=2.3')
    axes[2, 0].set_title('Power-Law Exponent (γ) by Country')
    axes[2, 0].set_xlabel('Country')
    axes[2, 0].set_ylabel('γ (Power-law exponent)')
    axes[2, 0].legend()
    axes[2, 0].tick_params(axis='x', rotation=45)

# 8. Validation compatibility scores
compatibility_data = [(result.country_name, result.iusmorfos_compatibility) 
                     for result in validation_results.values()]
comp_df = pd.DataFrame(compatibility_data, columns=['country', 'compatibility'])

colors = ['green' if score >= 0.7 else 'orange' if score >= 0.5 else 'red' 
          for score in comp_df['compatibility']]

axes[2, 1].bar(comp_df['country'], comp_df['compatibility'], color=colors, alpha=0.7)
axes[2, 1].axhline(y=0.7, color='green', linestyle='--', label='Validation threshold')
axes[2, 1].set_title('Iusmorfos Compatibility Score by Country')
axes[2, 1].set_xlabel('Country')
axes[2, 1].set_ylabel('Compatibility Score')
axes[2, 1].legend()
axes[2, 1].tick_params(axis='x', rotation=45)

# 9. Reform type distribution by legal tradition
reform_tradition = pd.crosstab(analysis_df['reform_type'], analysis_df['legal_tradition'], 
                              normalize='columns')
reform_tradition.plot(kind='bar', ax=axes[2, 2], stacked=True)
axes[2, 2].set_title('Reform Type Distribution by Legal Tradition')
axes[2, 2].set_xlabel('Reform Type')
axes[2, 2].set_ylabel('Proportion')
axes[2, 2].legend(title='Legal Tradition', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[2, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Statistical summary
print(f"\n📊 Cross-Country Statistical Summary:")
print(f"\nBy Legal Tradition:")
for tradition, data in cross_analysis['by_legal_tradition'].items():
    print(f"  {tradition}:")
    print(f"    Countries: {', '.join(data['countries'])}")
    print(f"    Validation rate: {data['validation_rate']:.1%}")
    print(f"    Mean compatibility: {data['mean_compatibility']:.3f}")

print(f"\nBy Development Level:")
for level, data in cross_analysis['by_development_level'].items():
    print(f"  {level}:")
    print(f"    Countries: {', '.join(data['countries'])}")
    print(f"    Validation rate: {data['validation_rate']:.1%}")
    print(f"    Mean compatibility: {data['mean_compatibility']:.3f}")

## 5. Legal Tradition Analysis {#legal-tradition}

Deep dive into how legal traditions affect framework applicability.

In [ ]:
print("⚖️ LEGAL TRADITION ANALYSIS")
print("=" * 35)

# Group data by legal tradition
tradition_groups = {
    tradition: analysis_df[analysis_df['legal_tradition'] == tradition]
    for tradition in analysis_df['legal_tradition'].unique()
}

print(f"Legal traditions analyzed: {list(tradition_groups.keys())}")

# Statistical tests between legal traditions
traditions = list(tradition_groups.keys())
if len(traditions) >= 2:
    
    # Test complexity differences
    complexity_groups = [group['complexity_score'].values for group in tradition_groups.values()]
    if all(len(group) > 0 for group in complexity_groups):
        h_stat, p_value = kruskal(*complexity_groups)
        print(f"\n🧪 Complexity by Legal Tradition:")
        print(f"Kruskal-Wallis H = {h_stat:.3f}, p = {p_value:.4f}")
        if p_value < 0.05:
            print("✅ Significant differences in complexity between legal traditions")
        else:
            print("❌ No significant differences in complexity")
    
    # Test adoption success differences
    adoption_groups = [group['adoption_success'].values for group in tradition_groups.values()]
    if all(len(group) > 0 for group in adoption_groups):
        h_stat, p_value = kruskal(*adoption_groups)
        print(f"\n🧪 Adoption Success by Legal Tradition:")
        print(f"Kruskal-Wallis H = {h_stat:.3f}, p = {p_value:.4f}")
        if p_value < 0.05:
            print("✅ Significant differences in adoption success between legal traditions")
        else:
            print("❌ No significant differences in adoption success")

# Detailed tradition analysis
tradition_summary = analysis_df.groupby('legal_tradition').agg({
    'complexity_score': ['count', 'mean', 'std', 'median'],
    'adoption_success': ['mean', 'std', 'median'],
    'citation_count': ['sum', 'mean', 'median'],
    'implementation_months': ['mean', 'median'],
    'during_crisis': 'sum'
}).round(3)

print(f"\n📊 Legal Tradition Summary Statistics:")
print(tradition_summary)

# Create tradition-specific visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Complexity distribution by tradition
for i, (tradition, data) in enumerate(tradition_groups.items()):
    axes[0, 0].hist(data['complexity_score'], alpha=0.6, label=tradition, bins=20)
axes[0, 0].set_title('Complexity Score Distribution by Legal Tradition')
axes[0, 0].set_xlabel('Complexity Score')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Adoption success by tradition
tradition_adoption = [data['adoption_success'].values for data in tradition_groups.values()]
tradition_labels = list(tradition_groups.keys())

axes[0, 1].boxplot(tradition_adoption, labels=tradition_labels)
axes[0, 1].set_title('Adoption Success by Legal Tradition')
axes[0, 1].set_xlabel('Legal Tradition')
axes[0, 1].set_ylabel('Adoption Success Rate')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# 3. Implementation time by tradition
tradition_implementation = [data['implementation_months'].values for data in tradition_groups.values()]

axes[1, 0].boxplot(tradition_implementation, labels=tradition_labels)
axes[1, 0].set_title('Implementation Time by Legal Tradition')
axes[1, 0].set_xlabel('Legal Tradition')
axes[1, 0].set_ylabel('Implementation Time (months)')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# 4. Crisis responsiveness by tradition
crisis_by_tradition = analysis_df.groupby('legal_tradition')['during_crisis'].agg(['sum', 'count'])
crisis_by_tradition['crisis_rate'] = crisis_by_tradition['sum'] / crisis_by_tradition['count']

axes[1, 1].bar(crisis_by_tradition.index, crisis_by_tradition['crisis_rate'], alpha=0.7)
axes[1, 1].set_title('Crisis Period Innovation Rate by Legal Tradition')
axes[1, 1].set_xlabel('Legal Tradition')
axes[1, 1].set_ylabel('Proportion of Innovations During Crises')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tradition-specific insights
print(f"\n🔍 Key Insights by Legal Tradition:")

for tradition, data in tradition_groups.items():
    countries = data['name'].unique()
    mean_complexity = data['complexity_score'].mean()
    mean_adoption = data['adoption_success'].mean()
    mean_citations = data['citation_count'].mean()
    
    print(f"\n{tradition}:")
    print(f"  Countries: {', '.join(countries)}")
    print(f"  Average complexity: {mean_complexity:.2f}")
    print(f"  Average adoption success: {mean_adoption:.3f}")
    print(f"  Average citations: {mean_citations:.1f}")
    
    # Validation rates for this tradition
    tradition_validation = cross_analysis['by_legal_tradition'].get(tradition, {})
    if tradition_validation:
        print(f"  Validation rate: {tradition_validation['validation_rate']:.1%}")
        print(f"  Mean compatibility: {tradition_validation['mean_compatibility']:.3f}")

print(f"\n💡 Legal Tradition Conclusions:")
print(f"• Framework performance varies by legal tradition")
print(f"• Consider tradition-specific parameter adjustments")
print(f"• Mixed systems may require special handling")

## 6. Power-Law Universality Testing {#power-law}

Test whether the power-law hypothesis (γ≈2.3) holds across countries.

In [ ]:
print("📊 POWER-LAW UNIVERSALITY TESTING")
print("=" * 40)

# Analyze power-law patterns for each country
power_law_results = {}

for country_code, dataset in country_datasets.items():
    country_name = validator.country_metadata[country_code]['name']
    citations = dataset['citation_count'].values
    citations_nz = citations[citations > 0]
    
    if len(citations_nz) > 10:
        try:
            unique_citations, counts = np.unique(citations_nz, return_counts=True)
            if len(unique_citations) > 3:
                log_x = np.log(unique_citations)
                log_y = np.log(counts)
                
                slope, intercept, r_value, p_value, std_err = stats.linregress(log_x, log_y)
                gamma = -slope
                r_squared = r_value ** 2
                
                power_law_results[country_code] = {
                    'country_name': country_name,
                    'gamma': gamma,
                    'r_squared': r_squared,
                    'p_value': p_value,
                    'std_error': std_err,
                    'n_points': len(unique_citations),
                    'deviation_from_23': abs(gamma - 2.3),
                    'close_to_expected': abs(gamma - 2.3) <= 0.5
                }
                
        except Exception as e:
            print(f"Power-law analysis failed for {country_name}: {e}")

if power_law_results:
    pl_df = pd.DataFrame.from_dict(power_law_results, orient='index')
    
    print(f"\n📊 Power-Law Analysis Results:")
    print(pl_df[['country_name', 'gamma', 'r_squared', 'deviation_from_23', 'close_to_expected']].round(3))
    
    # Statistical tests
    gamma_values = pl_df['gamma'].values
    
    print(f"\n🧪 Statistical Tests:")
    print(f"Expected γ: 2.3")
    print(f"Observed γ: {gamma_values.mean():.3f} ± {gamma_values.std():.3f}")
    
    # One-sample t-test against expected value
    if len(gamma_values) > 1:
        t_stat, p_value = stats.ttest_1samp(gamma_values, 2.3)
        print(f"One-sample t-test: t = {t_stat:.3f}, p = {p_value:.4f}")
        
        if p_value > 0.05:
            print("✅ γ values consistent with expected 2.3")
        else:
            print("⚠️ γ values significantly different from expected 2.3")
    
    # Proportion close to expected
    close_proportion = pl_df['close_to_expected'].mean()
    print(f"\nCountries with γ close to 2.3 (±0.5): {close_proportion:.1%}")
    
    # Create power-law visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Plot individual country power-law fits
    plot_countries = list(power_law_results.keys())[:5]  # First 5 countries
    
    for i, country_code in enumerate(plot_countries):
        if i >= 5:
            break
            
        ax = axes[i // 3, i % 3]
        dataset = country_datasets[country_code]
        citations = dataset['citation_count'].values
        citations_nz = citations[citations > 0]
        
        if len(citations_nz) > 0:
            unique_citations, counts = np.unique(citations_nz, return_counts=True)
            
            if len(unique_citations) > 1:
                ax.loglog(unique_citations, counts, 'bo-', alpha=0.6, markersize=4, label='Data')
                
                # Fitted line
                result = power_law_results[country_code]
                x_fit = np.linspace(unique_citations.min(), unique_citations.max(), 50)
                # Using the regression: log(y) = intercept + slope * log(x)
                # So y = exp(intercept) * x^slope
                log_x = np.log(unique_citations)
                log_y = np.log(counts)
                slope, intercept, _, _, _ = stats.linregress(log_x, log_y)
                y_fit = np.exp(intercept) * x_fit ** slope
                
                ax.loglog(x_fit, y_fit, 'r--', alpha=0.8, 
                         label=f'γ = {result["gamma"]:.2f}')
                
                ax.set_title(f'{result["country_name"]}\n(R² = {result["r_squared"]:.3f})')
                ax.set_xlabel('Citation Count')
                ax.set_ylabel('Frequency')
                ax.legend()
                ax.grid(True, alpha=0.3)
    
    # Hide empty subplots
    for i in range(len(plot_countries), 6):
        axes[i // 3, i % 3].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Summary visualization
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # 1. Gamma values by country
    axes[0].bar(range(len(pl_df)), pl_df['gamma'], alpha=0.7, 
               color=['green' if close else 'orange' for close in pl_df['close_to_expected']])
    axes[0].axhline(y=2.3, color='red', linestyle='--', label='Expected γ=2.3')
    axes[0].axhline(y=2.3-0.5, color='red', linestyle=':', alpha=0.5, label='±0.5 range')
    axes[0].axhline(y=2.3+0.5, color='red', linestyle=':', alpha=0.5)
    axes[0].set_title('Power-Law Exponent (γ) by Country')
    axes[0].set_xlabel('Country')
    axes[0].set_ylabel('γ')
    axes[0].set_xticks(range(len(pl_df)))
    axes[0].set_xticklabels(pl_df['country_name'], rotation=45)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # 2. R-squared values
    axes[1].bar(range(len(pl_df)), pl_df['r_squared'], alpha=0.7, color='skyblue')
    axes[1].axhline(y=0.7, color='green', linestyle='--', label='Good fit threshold')
    axes[1].set_title('Power-Law Fit Quality (R²)')
    axes[1].set_xlabel('Country')
    axes[1].set_ylabel('R²')
    axes[1].set_xticks(range(len(pl_df)))
    axes[1].set_xticklabels(pl_df['country_name'], rotation=45)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # 3. Deviation from expected
    axes[2].bar(range(len(pl_df)), pl_df['deviation_from_23'], alpha=0.7, color='coral')
    axes[2].axhline(y=0.5, color='red', linestyle='--', label='Acceptable deviation')
    axes[2].set_title('Deviation from Expected γ=2.3')
    axes[2].set_xlabel('Country')
    axes[2].set_ylabel('|γ - 2.3|')
    axes[2].set_xticks(range(len(pl_df)))
    axes[2].set_xticklabels(pl_df['country_name'], rotation=45)
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n🎯 Power-Law Universality Assessment:")
    if close_proportion >= 0.8:
        print(f"✅ Strong universality - {close_proportion:.0%} of countries show expected power-law")
    elif close_proportion >= 0.6:
        print(f"📊 Moderate universality - {close_proportion:.0%} of countries show expected power-law")
    else:
        print(f"⚠️ Weak universality - only {close_proportion:.0%} of countries show expected power-law")
        
else:
    print("❌ Insufficient data for power-law analysis")

## 9. Framework Generalizability Assessment {#generalizability}

Overall assessment of Iusmorfos framework generalizability across countries.

In [ ]:
print("🌍 FRAMEWORK GENERALIZABILITY ASSESSMENT")
print("=" * 50)

# Generate comprehensive validation report
validation_report = validator.generate_validation_report()

# Extract key metrics
overall_validation_rate = validation_report['overall_assessment']['overall_validation_rate']
passed_countries = validation_report['overall_assessment']['validation_passed_countries']
failed_countries = validation_report['overall_assessment']['validation_failed_countries']

print(f"\n📊 OVERALL ASSESSMENT SUMMARY:")
print(f"Countries tested: {len(validation_results)}")
print(f"Validations passed: {len(passed_countries)}")
print(f"Validations failed: {len(failed_countries)}")
print(f"Overall success rate: {overall_validation_rate:.1%}")

print(f"\n✅ Countries passing validation:")
for country in passed_countries:
    print(f"  • {country}")

if failed_countries:
    print(f"\n❌ Countries failing validation:")
    for country in failed_countries:
        print(f"  • {country}")

# Detailed analysis by factors
print(f"\n🔍 DETAILED FACTOR ANALYSIS:")

# Factor 1: Legal Tradition Impact
tradition_analysis = cross_analysis['by_legal_tradition']
print(f"\n1. Legal Tradition Impact:")
for tradition, data in tradition_analysis.items():
    rate = data['validation_rate']
    compatibility = data['mean_compatibility']
    status = "✅" if rate >= 0.8 else "📊" if rate >= 0.6 else "⚠️"
    print(f"   {tradition}: {status} {rate:.1%} success (compatibility: {compatibility:.3f})")

best_tradition = max(tradition_analysis, key=lambda x: tradition_analysis[x]['validation_rate'])
worst_tradition = min(tradition_analysis, key=lambda x: tradition_analysis[x]['validation_rate'])
print(f"   → Best performing: {best_tradition}")
print(f"   → Needs improvement: {worst_tradition}")

# Factor 2: Development Level Impact
development_analysis = cross_analysis['by_development_level']
print(f"\n2. Development Level Impact:")
for level, data in development_analysis.items():
    rate = data['validation_rate']
    compatibility = data['mean_compatibility']
    status = "✅" if rate >= 0.8 else "📊" if rate >= 0.6 else "⚠️"
    print(f"   {level}: {status} {rate:.1%} success (compatibility: {compatibility:.3f})")

# Factor 3: Power-Law Consistency
if power_law_results:
    pl_consistency = pl_df['close_to_expected'].mean()
    print(f"\n3. Power-Law Consistency:")
    print(f"   Countries with γ≈2.3: {pl_consistency:.1%}")
    print(f"   Mean γ: {pl_df['gamma'].mean():.3f} ± {pl_df['gamma'].std():.3f}")
    
    if pl_consistency >= 0.8:
        print(f"   ✅ Strong power-law universality")
    elif pl_consistency >= 0.6:
        print(f"   📊 Moderate power-law consistency")
    else:
        print(f"   ⚠️ Weak power-law consistency")

# Factor 4: Institutional Quality Correlation
if 'correlation_analysis' in cross_analysis:
    significant_correlations = cross_analysis['correlation_analysis']['significant_correlations']
    print(f"\n4. Institutional Quality Correlations:")
    if significant_correlations:
        for corr in significant_correlations:
            print(f"   {corr['variable']}: {corr['correlation']:+.3f} ({corr['strength']} correlation)")
    else:
        print(f"   No strong correlations found")

# Create comprehensive summary visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Overall validation success
success_data = [len(passed_countries), len(failed_countries)]
labels = ['Passed', 'Failed']
colors = ['green', 'red']

axes[0, 0].pie(success_data, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[0, 0].set_title(f'Overall Validation Results\n({overall_validation_rate:.1%} Success Rate)')

# 2. Validation by legal tradition
tradition_rates = [data['validation_rate'] for data in tradition_analysis.values()]
tradition_names = list(tradition_analysis.keys())

bars = axes[0, 1].bar(tradition_names, tradition_rates, alpha=0.7, 
                     color=['green' if rate >= 0.8 else 'orange' if rate >= 0.6 else 'red' 
                            for rate in tradition_rates])
axes[0, 1].set_title('Validation Success by Legal Tradition')
axes[0, 1].set_ylabel('Success Rate')
axes[0, 1].set_ylim(0, 1)
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# Add percentage labels on bars
for bar, rate in zip(bars, tradition_rates):
    axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                   f'{rate:.1%}', ha='center', va='bottom')

# 3. Compatibility scores by country
countries = [result.country_name for result in validation_results.values()]
compatibility_scores = [result.iusmorfos_compatibility for result in validation_results.values()]

colors_comp = ['green' if score >= 0.7 else 'orange' if score >= 0.5 else 'red' 
               for score in compatibility_scores]

bars = axes[1, 0].bar(countries, compatibility_scores, color=colors_comp, alpha=0.7)
axes[1, 0].axhline(y=0.7, color='green', linestyle='--', label='Validation threshold')
axes[1, 0].set_title('Iusmorfos Compatibility by Country')
axes[1, 0].set_ylabel('Compatibility Score')
axes[1, 0].set_ylim(0, 1)
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Power-law gamma distribution
if power_law_results:
    gamma_values = pl_df['gamma'].values
    axes[1, 1].hist(gamma_values, bins=10, alpha=0.7, edgecolor='black', color='skyblue')
    axes[1, 1].axvline(x=2.3, color='red', linestyle='--', linewidth=2, label='Expected γ=2.3')
    axes[1, 1].axvline(x=gamma_values.mean(), color='orange', linestyle='-', linewidth=2, 
                      label=f'Observed mean: {gamma_values.mean():.2f}')
    axes[1, 1].set_title('Distribution of Power-Law Exponents')
    axes[1, 1].set_xlabel('γ (Power-law exponent)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'Insufficient power-law data', 
                   transform=axes[1, 1].transAxes, ha='center', va='center')
    axes[1, 1].set_title('Power-Law Analysis Unavailable')

plt.tight_layout()
plt.show()

# Final generalizability assessment
print(f"\n🎯 FINAL GENERALIZABILITY ASSESSMENT:")

if overall_validation_rate >= 0.8:
    assessment = "✅ STRONG GENERALIZABILITY"
    confidence = "High"
elif overall_validation_rate >= 0.6:
    assessment = "📊 MODERATE GENERALIZABILITY"
    confidence = "Medium"
else:
    assessment = "⚠️ WEAK GENERALIZABILITY"
    confidence = "Low"

print(f"\nOverall Assessment: {assessment}")
print(f"Confidence Level: {confidence}")
print(f"Success Rate: {overall_validation_rate:.1%}")

# Recommendations
print(f"\n💡 RECOMMENDATIONS:")
recommendations = validation_report.get('recommendations', [])
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

# Save validation report
report_path = validator.save_validation_results()
print(f"\n💾 Detailed validation report saved: {report_path}")

print(f"\n✅ Cross-country validation analysis complete!")